# 01_scrap
- scrapping data from thaigemjewelry.or.th/member and upload to DataPlatform

## Selenium login process

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import bs4 
import requests
import pandas as pd
from bs4 import BeautifulSoup

username = "ikwamkid@outlook.com"
password = "12345678"

driver = webdriver.Chrome(executable_path=r'C:\Users\Admin\Documents\GitHub\tryme_selenium\chromedriver\chromedriver')
driver.get('https://www.thaigemjewelry.or.th/member?page=1')

# กดปุ่ม log in
login = driver.find_element_by_xpath('/html/body/div[1]/div[2]/div[2]/a').click()

# ป้อน username/password
time.sleep(2)
user = driver.find_element_by_xpath('/html/body/div[3]/div/div/div[2]/form/div/div[1]/input')
pas = driver.find_element_by_xpath('/html/body/div[3]/div/div/div[2]/form/div/div[2]/input')
time.sleep(2)
user.send_keys(username)
pas.send_keys(password + Keys.ENTER)

# กลับไปกดเพจ member directory
time.sleep(2)
member = driver.find_element_by_xpath('/html/body/nav/div/div[3]/ul/li[3]/a').click()
memberdir = driver.find_element_by_xpath('/html/body/nav/div/div[3]/ul/li[3]/ul/li[1]/a').click()

#Zoom out เพื่อให้ load ทั่วหน้าจอ
driver.execute_script("document.body.style.Transform='scale(0.1)';")
driver.execute_script('document.body.style.TransformOrigin = "0 0";')

# ปิด cookie policy
cookie_button = driver.find_element_by_xpath('/html/body/div[20]/button').click()

## Scraping Process

In [25]:
name = []
product_type = []
bussiness_type = []

for i in range(77):
    url = "https://www.thaigemjewelry.or.th/member?page=" + str(i+1)
    driver.get(url)

    #shop = driver.find_elements_by_xpath('/html/body/section/div/div[3]/div/div/div[2]/div')
    for nm in driver.find_elements_by_css_selector('body > section > div > div.row > div > div > div.w-50.position-relative.member-text > div > h5'):
        name.append(nm.text)

    for pt in driver.find_elements_by_css_selector('body > section > div > div.row > div > div > div.w-50.position-relative.member-text > div > div.cate-type.mb-2 > p'):
        product_type.append(pt.text)

    for bt in driver.find_elements_by_css_selector('body > section > div > div.row > div > div > div.w-50.position-relative.member-text > div > div:nth-child(3) > p'):
        bussiness_type.append(bt.text)

driver.quit() 

In [27]:
table = pd.DataFrame([name,product_type,bussiness_type]).transpose()
table.columns = ['name','product_type','bussiness_type']
table

,name,product_type,bussiness_type
0,"PAUL WILD (THAILAND) CO.,LTD.",Gemstones,"Wholesaler, Exporter"
1,"RESSIGEAC GEMS CO.,LTD.",Gemstones,"Wholesaler, Exporter"
2,"YU FENG JEWELRY CO.,LTD.",Gemstones,"Retailer, Wholesaler, Importer, Exporter"
3,"EMERALD MINES CO.,LTD.",Gemstones,"Retailer, Wholesaler, Importer, Exporter"
4,"LA SIAM GEMS INTER CO.,LTD.","Gemsotnes,Precious and Semi Precious Stones","Exporter, Importer, Wholesaler, Retailer"
...,...,...,...
1530,"THANUSA CO.,LTD.",None,None
1531,"PATCHARA GALLERY CO.,LTD.",None,None
1532,PWK JEWELRY COMPANY LIMITED,None,None
1533,THAI COLOR STONES TRADE ASSOCIATION,None,None


In [29]:
table.to_excel(r'C:\Users\Admin\Documents\GitHub\dsi310_group9\week03\thaigem.xlsx')

- dump data to json file

In [20]:
import json
thaigem = []

In [21]:
table = table.to_dict('records')
#print (table)

In [23]:
print (f"page:{len(table)}")

page:1535


In [24]:
with open('thaigem.json', 'w') as f:
    json.dump(table, f)

## Upload data to DataPlatform

In [ ]:
from dsmlibrary.datanode import DataNode

In [ ]:
dir_rawData_id = 87

In [ ]:
datanode = DataNode()

In [ ]:
datanode.upload_file(directory_id=dir_rawData_id, file_path='thaigem.json')